In [117]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay

from sklearn.ensemble import RandomForestClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
display(train.shape)
display(test.shape)

(891, 12)

(418, 11)

In [6]:
display(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
display(sorted(train['Fare'].unique()))
display(train['Cabin'].unique())
display(train['SibSp'].unique())
display(train[train['Survived']==1].groupby('SibSp')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('SibSp')[['Survived']].apply(lambda a: a.value_counts()))
display(train['Age'].describe())

[0.0,
 4.0125,
 5.0,
 6.2375,
 6.4375,
 6.45,
 6.4958,
 6.75,
 6.8583,
 6.95,
 6.975,
 7.0458,
 7.05,
 7.0542,
 7.125,
 7.1417,
 7.225,
 7.2292,
 7.25,
 7.3125,
 7.4958,
 7.5208,
 7.55,
 7.6292,
 7.65,
 7.725,
 7.7292,
 7.7333,
 7.7375,
 7.7417,
 7.75,
 7.775,
 7.7875,
 7.7958,
 7.8,
 7.8292,
 7.8542,
 7.875,
 7.8792,
 7.8875,
 7.8958,
 7.925,
 8.0292,
 8.05,
 8.1125,
 8.1375,
 8.1583,
 8.3,
 8.3625,
 8.4042,
 8.4333,
 8.4583,
 8.5167,
 8.6542,
 8.6625,
 8.6833,
 8.7125,
 8.85,
 9.0,
 9.2167,
 9.225,
 9.35,
 9.475,
 9.4833,
 9.5,
 9.5875,
 9.825,
 9.8375,
 9.8417,
 9.8458,
 10.1708,
 10.4625,
 10.5,
 10.5167,
 11.1333,
 11.2417,
 11.5,
 12.0,
 12.275,
 12.2875,
 12.35,
 12.475,
 12.525,
 12.65,
 12.875,
 13.0,
 13.4167,
 13.5,
 13.7917,
 13.8583,
 13.8625,
 14.0,
 14.1083,
 14.4,
 14.4542,
 14.4583,
 14.5,
 15.0,
 15.0458,
 15.05,
 15.1,
 15.2458,
 15.5,
 15.55,
 15.7417,
 15.75,
 15.85,
 15.9,
 16.0,
 16.1,
 16.7,
 17.4,
 17.8,
 18.0,
 18.75,
 18.7875,
 19.2583,
 19.5,
 19.9667,
 20.2

891

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

array([1, 0, 3, 4, 2, 5, 8])

Survived,1
SibSp,
0,210
1,112
2,13
3,4
4,3


Survived,0
SibSp,
0,398
1,97
2,15
3,12
4,15
5,5
8,7


count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [19]:
bins = [0, 100, 200, 300, 400, 500, 600]
labels = ['0-100', '100-200', '200-300', '300-400', '400-500', '500-600']
train['Fare Group'] = pd.cut(train['Fare'], bins=bins, labels=labels, right=False)
test['Fare Group'] = pd.cut(test['Fare'], bins=bins, labels=labels, right=False)
display(train[train['Survived']==1].groupby('Fare Group')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('Fare Group')[['Survived']].apply(lambda a: a.value_counts()))

/tmp/ipykernel_30/1936605116.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train[train['Survived']==1].groupby('Fare Group')[['Survived']].apply(lambda a: a.value_counts()))


Fare Group  Survived
0-100       1           303
100-200     1            25
200-300     1            11
500-600     1             3
Name: count, dtype: int64

/tmp/ipykernel_30/1936605116.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train[train['Survived']==0].groupby('Fare Group')[['Survived']].apply(lambda a: a.value_counts()))


Fare Group  Survived
0-100       0           535
100-200     0             8
200-300     0             6
Name: count, dtype: int64

In [112]:
display(train['Pclass'].unique())
display(train[train['Survived']==1].groupby('Pclass')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('Pclass')[['Survived']].apply(lambda a: a.value_counts()))

array([3, 1, 2])

Survived,1
Pclass,
1,136
2,87
3,119


Survived,0
Pclass,
1,80
2,97
3,372


In [217]:
display(train[train['Survived']==1].groupby(['Sex', 'Age'])[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby(['Sex', 'Age'])[['Survived']].apply(lambda a: a.value_counts()))

Survived      1
Sex    Age     
female 0.75   2
       1.00   2
       2.00   2
       3.00   1
       4.00   5
...          ..
male   52.00  1
       56.00  1
       60.00  1
       62.00  1
       80.00  1

[104 rows x 1 columns]

Survived     0
Sex    Age    
female 2.0   4
       3.0   1
       6.0   1
       8.0   1
       9.0   4
...         ..
male   66.0  1
       70.0  2
       70.5  1
       71.0  2
       74.0  1

[112 rows x 1 columns]

In [213]:
total_females_survived = train[train['Survived']==1].groupby('Sex')[['Survived']].apply(lambda a: a.value_counts()).loc['female'].values[0]
total_males_survived = train[train['Survived']==1].groupby('Sex')[['Survived']].apply(lambda a: a.value_counts()).loc['male'].values[0]

In [242]:
total_males_survived

109

In [216]:
train[train['Sex']=='male' && train['Sex']].apply(lambda a: a[''])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare Group,Age Group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0-100,20-30
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0-100,30-40
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,0-100,NaN
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0-100,50-60
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,0-100,0-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S,0-100,20-30
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S,0-100,20-30
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0-100,20-30
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0-100,20-30


In [218]:
display(train['Age'].describe())
display(train['Age'].unique())
# Define age bins
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80']

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

In [219]:
train['Age Group'] = pd.cut(train['Age'], bins=bins, labels=labels, right=False)
test['Age Group'] = pd.cut(test['Age'], bins=bins, labels=labels, right=False)
display(train[train['Survived']==1].groupby('Age Group')[['Survived']].apply(lambda a: a.value_counts()))
display(train[train['Survived']==0].groupby('Age Group')[['Survived']].apply(lambda a: a.value_counts()))
#display(test.head())

/tmp/ipykernel_30/285530096.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train[train['Survived']==1].groupby('Age Group')[['Survived']].apply(lambda a: a.value_counts()))


Age Group  Survived
0-10       1           38
10-20      1           41
20-30      1           77
30-40      1           73
40-50      1           34
50-60      1           20
60-70      1            6
Name: count, dtype: int64

/tmp/ipykernel_30/285530096.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train[train['Survived']==0].groupby('Age Group')[['Survived']].apply(lambda a: a.value_counts()))


Survived,0
Age Group,
0-10,24
10-20,61
20-30,143
30-40,94
40-50,55
50-60,28
60-70,13
70-80,6


In [253]:
display(train[train['Survived']==1].groupby(['Sex', 'Age Group'])[['Survived']].apply(lambda a: a.value_counts()).loc['male'].apply(lambda x : (x / total_males_survived)).to_dict())
display(train[train['Survived']==1].groupby(['Sex', 'Age Group'])[['Survived']].apply(lambda a: a.value_counts()).loc['male'])

/tmp/ipykernel_30/458493117.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train[train['Survived']==1].groupby(['Sex', 'Age Group'])[['Survived']].apply(lambda a: a.value_counts()).loc['male'].apply(lambda x : (x / total_males_survived)).to_dict())


{(1,): {'0-10': 0.1743119266055046,
  '10-20': 0.06422018348623854,
  '20-30': 0.22935779816513763,
  '30-40': 0.21100917431192662,
  '40-50': 0.11009174311926606,
  '50-60': 0.03669724770642202,
  '60-70': 0.01834862385321101,
  '70-80': nan}}

/tmp/ipykernel_30/458493117.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train[train['Survived']==1].groupby(['Sex', 'Age Group'])[['Survived']].apply(lambda a: a.value_counts()).loc['male'])


Survived,1
Age Group,
0-10,19.0
10-20,7.0
20-30,25.0
30-40,23.0
40-50,12.0
50-60,4.0
60-70,2.0
70-80,NaN


In [24]:

display(train['Embarked'].unique())
embarked_mapping = {
    'nan': 0,
     np.nan: 0,
    'S': 1,
    'C': 2,
    'Q': 3
}
train_plne = train.copy()
test_plne = test.copy()
train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
train_median_age = train_plne['Age'].median()
test_median_age = test_plne['Age'].median()

# Impute NaN values in the Age column with the median age
train_plne['Age'].fillna(train_median_age, inplace=True)
test_plne['Age'].fillna(test_median_age, inplace=True)
display(train_plne.head())
display(test_plne.head())

array(['S', 'C', 'Q', nan], dtype=object)

/tmp/ipykernel_30/854410865.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/854410865.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/854410865.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method wil

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare Group,Age Group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,1,0-100,20-30
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,2,0-100,30-40
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,0-100,20-30
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,1,0-100,30-40
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,1,0-100,30-40


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare Group,Age Group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,3,0-100,30-40
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,1,0-100,40-50
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,3,0-100,60-70
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,1,0-100,20-30
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,1,0-100,20-30


In [117]:
test['Embarked'].unique()

array(['Q', 'S', 'C'], dtype=object)

In [25]:
train_plne = pd.get_dummies(train_plne, columns=['Sex','Embarked','Pclass','Age Group', 'Fare Group','SibSp'], prefix=['Sex','Embarked','Pclass','Age Group', 'Fare Group','SibSp'], dtype=int)
test_plne = pd.get_dummies(test_plne, columns=['Sex','Embarked','Pclass', 'Age Group', 'Fare Group', 'SibSp'], prefix=['Sex','Embarked','Pclass','Age Group', 'Fare Group', 'SibSp'], dtype=int)

In [26]:
train_plne.head()

,PassengerId,Survived,Name,Age,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,...,Fare Group_300-400,Fare Group_400-500,Fare Group_500-600,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,1,0,"Braund, Mr. Owen Harris",22.0,0,A/5 21171,7.2500,NaN,0,1,...,0,0,0,0,1,0,0,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,0,PC 17599,71.2833,C85,1,0,...,0,0,0,0,1,0,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,STON/O2. 3101282,7.9250,NaN,1,0,...,0,0,0,1,0,0,0,0,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,0,113803,53.1000,C123,1,0,...,0,0,0,0,1,0,0,0,0,0
4,5,0,"Allen, Mr. William Henry",35.0,0,373450,8.0500,NaN,0,1,...,0,0,0,1,0,0,0,0,0,0


In [27]:
nan_counts = train_plne.isna().sum()
display(nan_counts)
nan_counts = test_plne.isna().sum()
display(nan_counts)

PassengerId             0
Survived                0
Name                    0
Age                     0
Parch                   0
Ticket                  0
Fare                    0
Cabin                 687
Sex_female              0
Sex_male                0
Embarked_0              0
Embarked_1              0
Embarked_2              0
Embarked_3              0
Pclass_1                0
Pclass_2                0
Pclass_3                0
Age Group_0-10          0
Age Group_10-20         0
Age Group_20-30         0
Age Group_30-40         0
Age Group_40-50         0
Age Group_50-60         0
Age Group_60-70         0
Age Group_70-80         0
Fare Group_0-100        0
Fare Group_100-200      0
Fare Group_200-300      0
Fare Group_300-400      0
Fare Group_400-500      0
Fare Group_500-600      0
SibSp_0                 0
SibSp_1                 0
SibSp_2                 0
SibSp_3                 0
SibSp_4                 0
SibSp_5                 0
SibSp_8                 0
dtype: int64

PassengerId             0
Name                    0
Age                     0
Parch                   0
Ticket                  0
Fare                    1
Cabin                 327
Sex_female              0
Sex_male                0
Embarked_1              0
Embarked_2              0
Embarked_3              0
Pclass_1                0
Pclass_2                0
Pclass_3                0
Age Group_0-10          0
Age Group_10-20         0
Age Group_20-30         0
Age Group_30-40         0
Age Group_40-50         0
Age Group_50-60         0
Age Group_60-70         0
Age Group_70-80         0
Fare Group_0-100        0
Fare Group_100-200      0
Fare Group_200-300      0
Fare Group_300-400      0
Fare Group_400-500      0
Fare Group_500-600      0
SibSp_0                 0
SibSp_1                 0
SibSp_2                 0
SibSp_3                 0
SibSp_4                 0
SibSp_5                 0
SibSp_8                 0
dtype: int64

In [28]:
train_plne.dtypes

PassengerId             int64
Survived                int64
Name                   object
Age                   float64
Parch                   int64
Ticket                 object
Fare                  float64
Cabin                  object
Sex_female              int64
Sex_male                int64
Embarked_0              int64
Embarked_1              int64
Embarked_2              int64
Embarked_3              int64
Pclass_1                int64
Pclass_2                int64
Pclass_3                int64
Age Group_0-10          int64
Age Group_10-20         int64
Age Group_20-30         int64
Age Group_30-40         int64
Age Group_40-50         int64
Age Group_50-60         int64
Age Group_60-70         int64
Age Group_70-80         int64
Fare Group_0-100        int64
Fare Group_100-200      int64
Fare Group_200-300      int64
Fare Group_300-400      int64
Fare Group_400-500      int64
Fare Group_500-600      int64
SibSp_0                 int64
SibSp_1                 int64
SibSp_2   

Check if the age group is a feature that can be used, dummy variable
Cabin seems to have 40% values missing but maybe useful to show likelihood of survival, can be a dummy variable
Maybe PClass should be a dummy variable
We should also look at the fares and see if passengers with higher or lower fare had more probability of survival maybe can be dummy variable based on range.

In [29]:
train_plne.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Sex_female', 'Sex_male', 'Embarked_0', 'Embarked_1',
       'Embarked_2', 'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age Group_0-10', 'Age Group_10-20', 'Age Group_20-30',
       'Age Group_30-40', 'Age Group_40-50', 'Age Group_50-60',
       'Age Group_60-70', 'Age Group_70-80', 'Fare Group_0-100',
       'Fare Group_100-200', 'Fare Group_200-300', 'Fare Group_300-400',
       'Fare Group_400-500', 'Fare Group_500-600', 'SibSp_0', 'SibSp_1',
       'SibSp_2', 'SibSp_3', 'SibSp_4', 'SibSp_5', 'SibSp_8'],
      dtype='object')

In [30]:
drop_columns_train = ['Name','Age','Parch','Fare','Ticket','Cabin','Embarked_0']
drop_columns_test = ['Name','Age','Parch','Fare','Ticket','Cabin']
train_plne.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Sex_female', 'Sex_male', 'Embarked_0', 'Embarked_1',
       'Embarked_2', 'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age Group_0-10', 'Age Group_10-20', 'Age Group_20-30',
       'Age Group_30-40', 'Age Group_40-50', 'Age Group_50-60',
       'Age Group_60-70', 'Age Group_70-80', 'Fare Group_0-100',
       'Fare Group_100-200', 'Fare Group_200-300', 'Fare Group_300-400',
       'Fare Group_400-500', 'Fare Group_500-600', 'SibSp_0', 'SibSp_1',
       'SibSp_2', 'SibSp_3', 'SibSp_4', 'SibSp_5', 'SibSp_8'],
      dtype='object')

In [193]:
import pandas as pd
train_num = train_plne.drop(drop_columns_train, axis=1)
test_num = test_plne.drop(drop_columns_test, axis=1)
X = train_num.drop(['Survived','PassengerId'], axis=1)
y = train_num['Survived'].to_frame()

In [32]:
print(X.columns)
print(test_num.columns)

Index(['Sex_female', 'Sex_male', 'Embarked_1', 'Embarked_2', 'Embarked_3',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Age Group_0-10', 'Age Group_10-20',
       'Age Group_20-30', 'Age Group_30-40', 'Age Group_40-50',
       'Age Group_50-60', 'Age Group_60-70', 'Age Group_70-80',
       'Fare Group_0-100', 'Fare Group_100-200', 'Fare Group_200-300',
       'Fare Group_300-400', 'Fare Group_400-500', 'Fare Group_500-600',
       'SibSp_0', 'SibSp_1', 'SibSp_2', 'SibSp_3', 'SibSp_4', 'SibSp_5',
       'SibSp_8'],
      dtype='object')
Index(['PassengerId', 'Sex_female', 'Sex_male', 'Embarked_1', 'Embarked_2',
       'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Age Group_0-10',
       'Age Group_10-20', 'Age Group_20-30', 'Age Group_30-40',
       'Age Group_40-50', 'Age Group_50-60', 'Age Group_60-70',
       'Age Group_70-80', 'Fare Group_0-100', 'Fare Group_100-200',
       'Fare Group_200-300', 'Fare Group_300-400', 'Fare Group_400-500',
       'Fare Group_500-600', 'SibSp_0',

In [33]:
import torch
xs = torch.Tensor(X.values)
ys = torch.Tensor(y.values)
xs[0]

tensor([0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])

In [173]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
# Create a TensorDataset and DataLoader
X_train, X_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2, random_state=42, shuffle=True)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False) #no need to shuffle validation.

In [174]:
len(xs[0])

29

In [175]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Second fully connected layer
        self.fc3 = nn.Linear(hidden_size, output_size) # Third fully connected layer
        self.sigmoid = nn.Sigmoid() #Sigmoid activation for binary classification

    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

# Instantiate the model
input_size = len(xs[0])
hidden_size = len(xs[0]) * 2
output_size = 1  # Binary classification, so one output neuron
model = SimpleNN(input_size, hidden_size, output_size)

In [176]:
import torch.optim as optim
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [177]:
epochs = 1000

for epoch in range(epochs):
    # Training
    model.train()  # Set the model to training mode
    for inputs, labels in train_dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).squeeze(0) #squeeze to remove extra dimension.

        # Calculate the loss
        loss = criterion(outputs, labels.squeeze(0)) #convert labels to float

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculations during validation
        for inputs, labels in val_dataloader:
            outputs = model(inputs).squeeze(0)
            val_loss += criterion(outputs, labels.squeeze(0)).item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.squeeze(0)).sum().item()

    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = 100 * correct / total

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%')

print('Training finished!')

Epoch [10/1000], Train Loss: 0.4708, Val Loss: 0.4481, Accuracy: 80.45%
Epoch [20/1000], Train Loss: 0.5732, Val Loss: 0.4336, Accuracy: 81.56%
Epoch [30/1000], Train Loss: 0.1858, Val Loss: 0.4340, Accuracy: 82.12%
Epoch [40/1000], Train Loss: 0.5355, Val Loss: 0.4344, Accuracy: 81.56%
Epoch [50/1000], Train Loss: 0.8145, Val Loss: 0.4300, Accuracy: 80.45%
Epoch [60/1000], Train Loss: 0.3475, Val Loss: 0.4340, Accuracy: 81.56%
Epoch [70/1000], Train Loss: 0.5597, Val Loss: 0.4311, Accuracy: 81.56%
Epoch [80/1000], Train Loss: 0.3465, Val Loss: 0.4307, Accuracy: 81.56%
Epoch [90/1000], Train Loss: 0.7376, Val Loss: 0.4351, Accuracy: 80.45%
Epoch [100/1000], Train Loss: 0.3784, Val Loss: 0.4325, Accuracy: 81.56%
Epoch [110/1000], Train Loss: 0.4603, Val Loss: 0.4298, Accuracy: 81.01%
Epoch [120/1000], Train Loss: 0.5028, Val Loss: 0.4292, Accuracy: 81.56%
Epoch [130/1000], Train Loss: 0.4345, Val Loss: 0.4319, Accuracy: 80.45%
Epoch [140/1000], Train Loss: 0.9424, Val Loss: 0.4323, Accu

In [185]:
test_num.drop('PassengerId', axis=1)

,Sex_female,Sex_male,Embarked_1,Embarked_2,Embarked_3,Pclass_1,Pclass_2,Pclass_3,Age Group_0-10,Age Group_10-20,...,Fare Group_300-400,Fare Group_400-500,Fare Group_500-600,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8
0,0,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
414,1,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
415,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
416,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [197]:
X.values.ravel()

array([0, 1, 1, ..., 0, 0, 0])

In [203]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X.values, y.values)
predictions = model.predict(test_num.drop('PassengerId', axis=1).values)
output = pd.DataFrame({'PassengerId': test_num.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


/tmp/ipykernel_30/1135409607.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X.values, y.values)


In [204]:
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [178]:
model.eval()  # Set to evaluation mode

SimpleNN(
  (fc1): Linear(in_features=29, out_features=58, bias=True)
  (fc2): Linear(in_features=58, out_features=58, bias=True)
  (fc3): Linear(in_features=58, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [111]:
input_features = test_num.drop('PassengerId', axis=1).iloc[5].values
len(input_features)

29

In [112]:
import torch

# Example input data
new_input = torch.Tensor(input_features) 

# Make predictions
with torch.no_grad():
    predictions = model(new_input)

# Process the predictions
predicted_label = (predictions > 0.5).float()  # Binary classification example
print("Predictions:", predictions)
print("Predicted label:", predicted_label)

Predictions: tensor([0.0786])
Predicted label: tensor([0.])


In [113]:
test_dataloader = TensorDataset(torch.Tensor(test_num.drop('PassengerId', axis=1).values))

In [114]:
ids = []
predictions = []
with torch.no_grad():  # Disable gradient calculations
    for (inputs,id) in zip(test_dataloader, test['PassengerId'].values):
        inputs = inputs[0] #dataloader returns a tuple, so we need to grab the first element.
        outputs = model(inputs).squeeze(0)  # Get model outputs
        predicted = (outputs > 0.5).float()  # Apply threshold for binary classification
        predictions.append(predicted.item())  # Collect predictions

# --- Processing predictions ---
print("Predictions:", predictions)

Predictions: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0

In [115]:
ids = []
predictions = []
for (X,id) in zip(test_dataloader, test['PassengerId'].values):
    label = model(X[0])
    ids.append(id)
    predictions.append((int((label > 0.5).float().sum().item())))

In [116]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'PassengerId': ids,
    'Survived': predictions
})

# Save the DataFrame to a CSV file without the default index
df.to_csv('submission.csv', index=False)